# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [135]:
# general + data loading
import pandas as pd
from sqlalchemy import create_engine
from itertools import chain

# pre-processing
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.chunk import tree2conlltags
import re

# sklearn - general
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
# sklearn - classifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
# sklearn - metrics
from sklearn.metrics import f1_score, precision_score, recall_score
# sklearn - customs
from sklearn.base import BaseEstimator, TransformerMixin

# storage
import pickle
import time

In [121]:
# download relevant wordnet models
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('omw')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

True

In [37]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_data.db')
df = pd.read_sql_table('texts', engine)
df.head()
X = df[['message', 'original', 'genre']]
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [ ]:
# init these globally (to avoid call for every run)
stops = set(stopwords.words('english'))
wlem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    '''Matches the given treebank POS tag to a wordnet one to be used by lemmatizer.'''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def tokenize(text):
    '''Tokenizes the words in the text, uses lemmatization and POS tagging.'''
    # remove punctuation
    text = re.sub("[\.,\\:;!?'\"-]", " ", text.lower())
    tokens = word_tokenize(text)
    
    # remove stopwords
    tokens = filter(lambda x: x not in stops, tokens)
    
    # part of speech
    tags = map(lambda x: (wlem.lemmatize(x[0], pos=get_wordnet_pos(x[1])), x[1]), tokens)
    
    return list(tags)

def tokenize_clean(text):
    '''Removes POS Tags from the words.'''
    return list(map(lambda x: x[0], tokenize(text)))

def tokenize_ner(text):
    '''Applies Named Entity Recognition to the words.'''
    # remove punctuation
    text = re.sub("[\.,\\:;!?'\"-]", " ", text.lower())
    sents = sent_tokenize(text)
    
    tokens = chain.from_iterable(map(lambda x: tree2conlltags( ne_chunk(pos_tag(word_tokenize(x))) ), sents))
    tokens = filter(lambda x: x[0] not in stops, tokens)
    tokens = map(lambda x: (wlem.lemmatize(x[0], pos=get_wordnet_pos(x[1])), x[1], x[2]), tokens)
    
    return list(tokens)
    
#%timeit X['message'].head(20).apply(tokenize_ner).head()
X['message'].head(20).apply(tokenize_ner).head()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [180]:
# simple baseline model
pipeline = [
    ('vectorize', CountVectorizer(tokenizer=tokenize_clean)),
    ('tfidf', TfidfTransformer()),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
]
pipeline = Pipeline(pipeline)

Define a function to add the test results to the csv file:

In [ ]:
def score_and_doc(model, name, X_test, y_test):
    '''General scoring function.
    
    Takes the model and predicts output against given data.
    Finally scores them along different metrics and writes the results to the experiments file.
    
    Args:
        model: The sklearn model
        name (str): name of the model (used for documentation)
        X_test: test data
        y_test: expected test labels
    '''
    # predict the data
    y_pred = model.predict(X_test)
    
    # score the model
    avg = 'micro'
    f1 = f1_score(y_test, y_pred, average=avg)
    prec = precision_score(y_test, y_pred, average=avg)
    rec = recall_score(y_test, y_pred, average=avg)
    
    # retrieve current config data?
    config = str(model.get_params())
    
    # open file and append
    f=open("../experiments.csv", "a+")
    f.write("{},{:.6f},{:.6f},{:.6f},{},{}\n".format(name, f1, prec, rec, time.ctime(), config))
    f.close()
    
    # print output
    print("{}: F1={:.6f} Prec={:.6f} Rec={:.6f}".format(name, f1, prec, rec))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# create small (mobile) dataset
df_s = df.sample(2000)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(df_s[['message', 'original', 'genre']], df_s.iloc[:, 4:], test_size=0.3, random_state=42)

In [118]:
pipeline.fit(Xs_train['message'], ys_train)

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('cls',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [119]:
score_and_doc(pipeline, 'baseline_s', Xs_test['message'], ys_test)

baseline: F1=0.543388 Prec=0.780415 Rec=0.416799


### 6. Improve your model
Use grid search to find better parameters. 

In [181]:
parameters = {
    #'vectorize__tokenizer': (tokenize_clean, tokenize_ner),
    'vectorize__ngram_range': ((1, 1), (1, 2), (1, 4)),
    'vectorize__max_df': (0.5, 0.75, 1.0),
    'vectorize__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'cls__estimator__max_iter': [50, 100, 150],
    'cls__estimator__C': [0.5, 1.0, 2.0],
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)
cv.fit(Xs_train['message'], ys_train)

c:\programming\envs\datascience\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorize',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [182]:
print("Best Parameters:")
print(cv.best_params_)
score_and_doc(cv, 'baseline_v2_s', Xs_test['message'], ys_test)

Best Parameters:
{'cls__estimator__max_iter': 50, 'cls__estimator__C': 2.0, 'tfidf__use_idf': False, 'vectorize__max_features': None, 'vectorize__max_df': 0.5, 'vectorize__ngram_range': (1, 1)}
baseline_v2_s: F1=0.604456 Prec=0.777409 Rec=0.494453


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Input Feature Experiments

Various Experiments regarding different input features for the model. Those input features are tested separately and then in combined fashion.

In [114]:
# added Text Length as feature
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

pipeline_length = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_length.fit(Xs_train['message'], ys_train)
score_and_doc(pipeline_length, 'feat_length_s', Xs_test['message'], ys_test)

feat_length_s: F1=0.532918 Prec=0.774421 Rec=0.406233


In [126]:
# added Starting Verb
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = tokenize(sentence)
            if len(pos_tags) == 0: continue
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

pipeline_verb = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_verb.fit(Xs_train['message'], ys_train)
score_and_doc(pipeline_verb, 'feat_verb_s', Xs_test['message'], ys_test)

feat_verb_s: F1=0.527197 Prec=0.775385 Rec=0.399366


In [178]:
# use the genre as additional feature
pipeline_genre = Pipeline([
    ('features',
        ColumnTransformer(transformers=[
            ('text', Pipeline([
                ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
                ('tfidf', TfidfTransformer())
            ]), 'message'),
            ('genre', Pipeline([
                ('category', OneHotEncoder())
            ]), ['genre'])
        ]),
    ),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_genre.fit(Xs_train, ys_train)
score_and_doc(pipeline_genre, 'feat_genre_s', Xs_test, ys_test)

feat_genre_s: F1=0.550017 Prec=0.767992 Rec=0.428420


**GloVe**

The next feature we are going to test is GloVe (Global Vectors for Word Representations). (GloVe is based on a factorization of co-occurence matrix to a lower dimensional space, while word2vec achieves a similar dimensionality reduction through a FF-NN).

We will use the pre-trained twitter vector-model here, under the assumption that it best captures the information in the message format of the datasets.

In [ ]:
# define glove estimator
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../models')
import glove
import importlib
importlib.reload(glove)

# testing multiple params of glove - (25, 'sent', None), (25, 'sent-matrix', 10), (25, 'sent-matrix', 20), 
for dim, agg_type, max_feat in [(25, 'centroid', 5), (25, 'centroid', 15), (100, 'sent', None)]:
    print("starting {} - {}".format(dim, agg_type))
    pipeline_glove = Pipeline([
        ('glove', glove.GloVeTransformer('twitter', dim, agg_type, tokenizer=tokenize_clean, max_feat=max_feat)),
        ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
    ])

    pipeline_glove.fit(X_train['message'], y_train)
    score_and_doc(pipeline_glove, 'feat_glove_{}d_{}'.format(dim, agg_type), X_test['message'], y_test)

starting 25 - centroid


c:\programming\envs\datascience\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
c:\programming\envs\datascience\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
c:\programming\envs\datascience\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
c:\programming\envs\datascience\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
c:\programming\envs\datascience\lib\site-packages\sklearn\cluster\k_means_.p

In [ ]:
# compose a feature union
pipeline_glove = Pipeline([
    ('features', FeatureUnion([
        ('term_emb', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('glove', Pipeline([
            ('glove', glove.GloVeTransformer('twitter', 100, 'centroid', tokenizer=tokenize_clean, max_feat=5))
        ]))
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])

pipeline_glove.fit(X_train['message'], y_train)
score_and_doc(pipeline_glove, 'feat_glove_tfidf_compose', X_test['message'], y_test)

**Putting it all together**

As we have a general understanding about the impact of the different features, lets try to combine the ones that delivered the best results into a single combined feature-set:

In [ ]:
# modularize the creation process
def create_pipeline(cls):
    '''Creates the feature pipeline with the specified classifier'''
    return Pipeline([
        ('features', FeatureUnion([
            ('text', Pipeline([
                ('message', FunctionTransformer(lambda x: x['message'], validate=False))
                ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
                ('tfidf', TfidfTransformer()),
            ])),
            ('genre', Pipeline([
                ('category', FunctionTransformer(lambda x: np.where(x['genre'] == m)[0][0]))
            ]))
        ])),
        ('cls',  cls)
    ])

# create the pipeline
pipeline_feat = create_pipeline( MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
# fit and score
pipeline_feat.fit(X_train[['message', 'genre']], y_train)
score_and_doc(pipeline_feat, 'feat_combined', X_test[['message', 'genre']], y_test)

Perform an additional grid search over the parameters (focused on the weightings, etc. of the features):

In [ ]:
parameters = {
    #'vectorize__tokenizer': (tokenize_clean, tokenize_ner),
    'vectorize__ngram_range': ((1, 1), (1, 2), (1, 4)),
    'vectorize__max_df': (0.5, 0.75, 1.0),
    'vectorize__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    #'cls__estimator__max_iter': [50, 100, 150],
    #'cls__estimator__C': [0.5, 1.0, 2.0],
}

cv = GridSearchCV(pipeline_feat, parameters, n_jobs=-1)
cv.fit(Xs_train['message'], ys_train)

In [ ]:
print("Best Parameters:")
print(cv.best_params_)
score_and_doc(cv, 'feat_combined_v2', X_test[['message', 'genre']], y_test)

#### Classifier Experiments

Now that we have specified our features to use, lets experiment with various classifiers

In [ ]:
def create_and_score(cls, name):
    '''Combines creation and scoring functions'''
    pipeline = create_pipeline(cls)
    pipeline.fit(X_train[['message', 'genre']], y_train)
    score_and_doc(pipeline, name, X_test[['message', 'genre']], y_test)
    return pipeline

In [ ]:
# decision tree
cls = MultiOutputClassifier(DecisionTreeClassifier(), n_jobs=-1)
pipeline_dt = create_and_score(cls, 'decision_tree')

# random forest
cls = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)
pipeline_rf = create_and_score(cls, 'random_forest')

In [ ]:
# Support Vector Classifier (might take some while)
cls = MultiOutputClassifier(SVC(), n_jobs=-1)
pipeline_sv = create_and_score(cls, 'support_vector')

In [ ]:
# Simple NN

# TODO: use skflow for that

In [ ]:
# RNN


In [ ]:
# Ensemble Methods


### 9. Export your model as a pickle file

In [21]:
# store the most promision pipeline
filename = '../models/pipeline.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.